In [73]:
import fiona
import dash
import geopandas as gpd
import simplejson as json
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import ast
import numpy as np
import sqlite3
import fb
import datetime
import sys
from geopandas.tools import sjoin
import warnings; warnings.simplefilter('ignore')
from sklearn.linear_model import LinearRegression
import  sys
import numpy as np
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import argparse
import csv
import collections
from sklearn.linear_model import LinearRegression
import geopandas as gpd
import pandas as pd
import os
import shutil
import plotly.graph_objects as go
from shapely.geometry import Polygon, Point
import pickle

In [74]:
def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

def get_db_fields(db_path, fields, local_tz=True):
    c = sqlite3.connect(db_path)
    c.row_factory = dict_factory

    D = {}
    dates = []

    #for row in c.execute('SELECT * FROM pop_tile'):
    for row in c.execute('SELECT * FROM pop_tile WHERE n_crisis != "\\N"'):
        lat = row['lat']
        lon = row['lon']
        if (lat,lon) not  in D:
                D[(lat,lon)] = {}

        date, time = row['date_time'].split()

        #if local_tz:
                #date, time = switch_tz(date,time)

        if date not in dates:
            dates.append(date)

        if date not in D[(lat,lon)]:
            D[(lat,lon)][date] = {}

        d = {}
        for field in fields:
            d[field] = row[field]

        D[(lat,lon)][date][time] = d
    return D

def get_crisis_vals(D, pos, dates_times, field):
	count = 0
	vals = []
	dates = []
	for date,time in dates_times:
		if date in D[pos]:
			if date not in dates:
				dates.append(date)
			if len(D[pos][date]) >= 3:
				if D[pos][date] not in vals:
					vals.append(D[pos][date])
			else:
				fill = {}
				count = 0
				for i in ['0000','0800','1600']:
					if i in D[pos][date]:
						x = {i: D[pos][date][i]}
						fill[i] = D[pos][date][i]

					else:
						count = count  +1 
						fill[i] = {'n_baseline': 'NA', 'n_crisis': 'NA'}

				if fill not in vals:
					vals.append(fill)
				
		else:
			return None, None

	return vals,dates

In [82]:
db = '/Users/DBurke/Documents/Layerlab/generalized_pipeline/dbs/boulder.db'
shapefile = '/Users/DBurke/Documents/Layerlab/generalized_pipeline/facebook/shapefiles/co_counties'
beg_doI = '20201214'
end_doI = '20210101'
sit_rep_name = 'Boulder'
base_path = '/Users/DBurke/Documents/Layerlab/generalized_pipeline/'
sit_rep_path = base_path+'sitreps/'+sit_rep_name+'/'+sit_rep_name +'_hot_spot_shapes' +'/'
location_data_path = base_path+ 'dash/location_data/'

In [83]:
c = sqlite3.connect(db)
c.row_factory = dict_factory
C = {}

for row in c.execute('SELECT lat,lon,n_crisis FROM pop_tile WHERE n_crisis != "\\N"'):
	lat = row['lat']
	lon = row['lon']
	if (lat,lon) not  in C:
		C[(lat,lon)] = []
	C[(lat,lon)].append(float(row['n_crisis']))
C_stats = []
for c in C:
	C_stats.append( ( np.mean(C[c]), np.std(C[c]) ) )
model = LinearRegression()
x=[[c[0]] for c in C_stats]
y=[[c[1]] for c in C_stats]
model.fit(x,y)


D = get_db_fields(db, ['n_baseline', 'n_crisis'])
gdf = gpd.read_file(shapefile)

In [84]:
dates_times = {}
for pos in D:
    for date in D[pos]:
        for time in D[pos][date]:
            if (date,time) not in dates_times:
                dates_times[(date,time)] = 1

dates_times = sorted(dates_times.keys())
if beg_doI and end_doI:
    new_dates_times = {}
    for date, time in dates_times:
        beg_obj = datetime.datetime.strptime(beg_doI, '%Y%m%d')
        end_obj = datetime.datetime.strptime(end_doI, '%Y%m%d')
        temp = datetime.datetime.strptime(date, '%Y-%m-%d')
        if temp >= beg_obj and temp <= end_obj:
            new_dates_times[(date,time)] = 1
    new_dates_times = sorted(new_dates_times.keys())
    dates_times = new_dates_times
print('done')

done


In [85]:
with open('pickle_test.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [86]:
df_dict_animation = b
df_dict = {}
for pos in D:
    lat = float(pos[0])
    lon = float(pos[1])
    position = pos[0] + ', ' + pos[1]
    #if lat >= 39.95 and lat <= 40.086 and lon >= -105.305 and lon <= -105.205: #boulder cut off 
    crisises, dates = get_crisis_vals(D, pos, dates_times, 'n_crisis')
    if lat >= 39.95 and lat <= 40.086 and lon >= -105.305 and lon <= -105.205: #boulder cut off
    #if lat >= 33.84 and lat <= 33.95 and lon >= 35.40 and lon <= 35.60:
        if crisises is not None:
            one = []
            two = []
            three = []
            for crisis in crisises:
                if crisis['0000']['n_crisis'] != 'NA':
                    one.append(crisis['0000']['n_crisis'])
                else:
                    one.append('NA')

                if crisis['0800']['n_crisis'] != 'NA':
                    two.append(crisis['0800']['n_crisis'])
                else:
                    two.append('NA')

                if crisis['1600']['n_crisis'] != 'NA':
                    three.append(crisis['1600']['n_crisis'])
                else:
                    three.append('NA')

            df = pd.DataFrame()
            df['dates'] = dates

            df['0000'] = one
            df['0000'] = df['0000'].replace({'NA': None})
            missings = df[df['0000'].isnull()].index.tolist()
            if len(missings) >= 12:
                df['0000'].drop
                continue
            else:
                if len(missings) > 0:
                    #for i in range(len(missings)): #loop thru missing indexes 
                    i = 0                        
                    while len(missings) != 0:
                        lower = missings[i] 
                        upper = missings[i]
                        low_gate = False
                        up_gate = False
                        if lower <= 0:
                            j = lower+1
                            while j in missings:
                                j = j + 1
                            print("j: ", j)
                            print("missings: ", missings)
                            print("i: ", i)
                            print("len of df: ", len(df['0000']))
                            df['0000'].iloc[missings[i]] = float(df['0000'].iloc[j])
                            missings.pop(i)
                            if len(missings) == 0:
                                break
                            low_gate = True

                        if upper >= len(missings):
                            j = upper-1
                            while j in missings:
                                j = j - 1
                            df['0000'].iloc[missings[i]] = float(df['0000'].iloc[j])
                            missings.pop(i)
                            if len(missings) == 0:
                                break
                            up_gate = True

                        if not low_gate and not up_gate:
                            lower = lower - 1
                            upper = upper + 1
                            if lower not in missings and upper not in missings:
                                avg = []
                                avg.append(float(df['0000'].iloc[lower]))
                                avg.append(float(df['0000'].iloc[upper]))
                                df['0000'].iloc[missings[i]] = np.mean(avg)
                                missings.pop(i)
                                if len(missings) == 0:
                                    break
                        i = i + 1
                        if i > len(missings)-1:
                            i = 0

            df['0800'] = two
            df['0800'] = df['0800'].replace({'NA': None})
            missings = df[df['0800'].isnull()].index.tolist()
            if len(missings) >= 12:
                df['0800'].drop
                continue
            else:
                if len(missings) > 0:
                    #for i in range(len(missings)): #loop thru missing indexes 
                    i = 0                        
                    while len(missings) != 0:
                        lower = missings[i] 
                        upper = missings[i]
                        low_gate = False
                        up_gate = False
                        if lower <= 0:
                            j = lower+1
                            while j in missings:
                                j = j + 1
                            df['0800'].iloc[missings[i]] = float(df['0800'].iloc[j])
                            missings.pop(i)
                            if len(missings) == 0:
                                break
                            low_gate = True

                        if upper >= len(missings):
                            j = upper-1
                            while j in missings:
                                j = j - 1
                            df['0800'].iloc[missings[i]] = float(df['0800'].iloc[j])
                            missings.pop(i)
                            if len(missings) == 0:
                                break
                            up_gate = True

                        if not low_gate and not up_gate:
                            lower = lower - 1
                            upper = upper + 1
                            if lower not in missings and upper not in missings:
                                avg = []
                                avg.append(float(df['0800'].iloc[lower]))
                                avg.append(float(df['0800'].iloc[upper]))
                                df['0800'].iloc[missings[i]] = np.mean(avg)
                                missings.pop(i)
                                if len(missings) == 0:
                                    break
                        i = i + 1
                        if i > len(missings)-1:
                            i = 0
            df['1600'] = three
            df['1600'] = df['1600'].replace({'NA': None})
            missings = df[df['1600'].isnull()].index.tolist()
            
            if len(missings) >= 12:
                df['1600'].drop
                continue
            else:
                if len(missings) > 0:
                    #for i in range(len(missings)): #loop thru missing indexes 
                    i = 0                        
                    while len(missings) != 0:
                        lower = missings[i] 
                        upper = missings[i]
                        low_gate = False
                        up_gate = False
                        if lower <= 0:
                            j = lower+1
                            while j in missings:
                                j = j + 1
                            df['1600'].iloc[missings[i]] = float(df['1600'].iloc[j])
                            missings.pop(i)
                            if len(missings) == 0:
                                break
                            low_gate = True

                        if upper >= len(missings):
                            j = upper-1
                            while j in missings:
                                j = j - 1
                            df['1600'].iloc[missings[i]] = float(df['1600'].iloc[j])
                            missings.pop(i)
                            if len(missings) == 0:
                                break
                            up_gate = True

                        if not low_gate and not up_gate:
                            lower = lower - 1
                            upper = upper + 1
                            if lower not in missings and upper not in missings:
                                avg = []
                                avg.append(float(df['1600'].iloc[lower]))
                                avg.append(float(df['1600'].iloc[upper]))
                                df['1600'].iloc[missings[i]] = np.mean(avg)
                                missings.pop(i)
                                if len(missings) == 0:
                                    break
                        i = i + 1
                        if i > len(missings)-1:
                            i = 0
        else:
            continue
        coordinates = df_dict_animation.loc[df_dict_animation['polygons'] == position]
        df['coordinates'] = coordinates['polygons']
        df_dict[position] = df
#print(df_dict['40.004475801858, -105.25177001953'])

In [87]:
print(len(df_dict))

104


In [81]:
with open('pickle_test_lines.pickle', 'wb') as handle:
    pickle.dump(df_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [65]:
from plotly.subplots import make_subplots
pickle_path = 'pickle_test_lines.pickle'
with open(pickle_path, 'rb') as handle:
    b = pickle.load(handle)
df_dict = b
dict_keys = []

map_pickle_path = 'pickle_test.pickle'
with open(map_pickle_path, 'rb') as handle:
    map_data = pickle.load(handle)

df_geo = map_data
#df_geo = pd.read_csv('/Users/DBurke/Documents/Layerlab/generalized_pipeline/dash/location_data/df_geo.csv')
with open('/Users/DBurke/Documents/Layerlab/generalized_pipeline/dash/location_data/json_geo.json') as data_file:
    new_jfile = json.load(data_file)

dict_keys = df_dict.keys()


fig = make_subplots(rows=3, cols=1, subplot_titles = ("0000 UTC", "0800 UTC", "1600 UTC"))

for key in df_dict.keys():
    if key in dict_keys:
        df = df_dict[key]
        df['0000'] = df['0000'].astype(float)
        df['0800'] = df['0800'].astype(float)
        df['1600'] = df['1600'].astype(float)
        dates = df['dates']
        
        c_0 = df['0000'].to_list()
        c_8 = df['0800'].to_list()
        c_16 = df['1600'].to_list()
        
        j = 0
        for c in [c_0, c_8, c_16]:
            result_mul = seasonal_decompose(c,
                                 period=3,
                                 model = 'multiplicative',
                                 extrapolate_trend='freq')
        
            s  = result_mul.seasonal
            t  = result_mul.trend
            t  = np.array(result_mul.trend)
            t = t - t[0]
            
            fig.add_trace(go.Scatter(x = dates, y = t, mode = 'lines'),row = j+1, col = 1)
            j = j + 1
        #fig.add_trace(go.Scatter(x = dates, y = df['0800'], mode = 'lines', showlegend = False),row = 2, col = 1)

        #fig.add_trace(go.Scatter(x = dates, y = df['1600'], mode = 'lines', showlegend = False),row = 3, col = 1)

fig.update_layout(template = 'plotly_dark')
fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_xaxes(title_text="Date", row=3, col=1)
fig.update_yaxes(title_text="Crisis Density", row=1, col=1)
fig.update_yaxes(title_text="Crisis Density", row=2, col=1)
fig.update_yaxes(title_text="Crisis Density", row=3, col=1)
fig.show()


In [22]:
pickle_path = 'pickle_test_lines.pickle'
map_pickle_path = 'pickle_test.pickle'
with open(pickle_path, 'rb') as handle:
    b = pickle.load(handle)
with open(map_pickle_path, 'rb') as handle:
    map_data = pickle.load(handle)
df_geo = map_data
df_dict = b
point_indexes = df_geo['ids']
point_indexes = [129]
cols = []
dict_keys = []
df = pd.DataFrame()
for point in point_indexes:
    df_temp = df_geo.loc[df_geo['ids']==int(point)]
    cols.append(df_temp['positions'].values[0])
print(cols)
x = df_geo.loc[df_geo['ids']==129]
print(x['z_score_2020-11-04_0000'])

['40.033923571574, -105.25177001953']
129    52.925388
Name: z_score_2020-11-04_0000, dtype: float64


In [23]:
print(df_dict)

{'-33.12145052824, -71.589660644531':          dates                0000                0800                1600  \
0   2021-01-15   37.58259588945021  27.997631803201333  31.813922511656255   
1   2021-01-16   38.30964759839541   35.90458378574106  37.750499421603166   
2   2021-01-17   37.84733251197456   35.74787867256562    39.7813099268739   
3   2021-01-18  39.076847373512635  28.973393442834887   32.19969865529876   
4   2021-01-19    38.2553782538584  29.180206418876477   32.48791815708618   
5   2021-01-20   37.94322078167013   28.74158839624709   30.47684101874613   
6   2021-01-21   37.74139429271879   29.98620762156344   32.45239405099143   
7   2021-01-22  37.797029313393466  30.196648931591938  33.462499833887115   
8   2021-01-23   38.67978250903022   35.27770442829443  37.665922646281004   
9   2021-01-24   38.63952278243051   35.07522553970596  38.232354685813995   
10  2021-01-25   38.46373884853574  30.193556305211295  33.789240626343975   
11  2021-01-26   38.909390